### Import lib

In [1]:
import jupyter_dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [2]:
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

### Import data

In [3]:
df = pd.read_csv("data/Berlin_crimes.csv")
df

,Year,District,Code,Location,Robbery,Street_robbery,Injury,Agg_assault,Threat,Theft,Car,From_car,Bike,Burglary,Fire,Arson,Damage,Graffiti,Drugs,Local
0,2012,Mitte,10111,Tiergarten Süd,70,46,586,194,118,2263,18,328,120,68,16,4,273,26,171,1032
1,2012,Mitte,10112,Regierungsviertel,65,29,474,123,142,3203,10,307,170,37,10,4,380,124,98,870
2,2012,Mitte,10113,Alexanderplatz,242,136,1541,454,304,8988,81,792,822,275,49,27,1538,522,435,3108
3,2012,Mitte,10114,Brunnenstraße Süd,52,25,254,60,66,1916,86,192,396,131,14,5,428,122,213,752
4,2012,Mitte,10221,Moabit West,130,51,629,185,199,2470,94,410,325,161,42,22,516,64,259,1403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2019,Reinickendorf,123012,Nord 2 - Waidmannslust/Wittenau/Lübars,34,19,372,85,123,1160,30,135,150,93,16,3,306,74,110,728
1196,2019,Reinickendorf,123021,MV 1 - Märkisches Viertel,42,22,491,123,187,1100,51,224,76,40,39,19,286,11,73,986
1197,2019,Reinickendorf,123022,MV 2 - Rollbergsiedlung,6,4,84,19,34,293,13,36,18,34,5,2,156,56,21,212
1198,2019,Reinickendorf,123043,West 3 - Borsigwalde/Freie Scholle,8,4,95,18,43,492,21,96,69,38,6,1,79,8,31,218


In [4]:
df = df.groupby('District')[['Street_robbery', 'Drugs']].median()
df

,Street_robbery,Drugs
District,,
Charlottenburg-Wilmersdorf,11.0,42.5
Friedrichshain-Kreuzberg,62.5,330.5
Lichtenberg,9.0,26.0
Marzahn-Hellersdorf,11.0,32.5
Mitte,45.0,231.0
Neukölln,30.0,55.0
Pankow,8.0,33.0
Reinickendorf,10.0,60.5
Spandau,10.0,29.5


### Layout

In [5]:
app.layout = html.Div([
        dbc.Row(dbc.Col(html.H3("Our Beautiful App Layout"),
                        width={'size': 6, 'offset': 3},
                        ),
                ),
        dbc.Row(dbc.Col(html.Div("One column is all we need because there ain't no room for the "
                                 "both of us in this raggedy town"),
                        width=4
                        )
                ),
        dbc.Row(
            [
                dbc.Col(dcc.Dropdown(id='c_dropdown', placeholder='last dropdown',
                                     options=[{'label': 'Option A', 'value': 'optA'},
                                              {'label': 'Option B', 'value': 'optB'}]),
                        width={'size': 3, "offset": 2, 'order': 3}
                        ),
                dbc.Col(dcc.Dropdown(id='a_dropdown', placeholder='first dropdown',
                                     options=[{'label': 'Option A', 'value': 'optA'},
                                              {'label': 'Option B', 'value': 'optB'}]),
                        width={'size': 4, "offset": 1, 'order': 1}
                        ),
                dbc.Col(dcc.Dropdown(id='b_dropdown', placeholder='middle dropdown',
                                     options=[{'label': 'Option A', 'value': 'optA'},
                                              {'label': 'Option B', 'value': 'optB'}]),
                        width={'size': 2,  "offset": 0, 'order': 2}
                        ),
            ], no_gutters=True
        ),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='pie_chart1', figure={}),
                        width=8, lg={'size': 6,  "offset": 0, 'order': 'first'}
                        ),
                dbc.Col(dcc.Graph(id='pie_chart2', figure={}),
                        width=4, lg={'size': 6,  "offset": 0, 'order': 'last'}
                        ),
            ]
        )
])

### Callback

In [6]:
@app.callback(
    [Output('pie_chart1', 'figure'),
     Output('pie_chart2', 'figure')],
    [Input('a_dropdown', 'value'),
     Input('b_dropdown', 'value'),
     Input('c_dropdown', 'value')]
)
def update_graph(dpdn_a, dpdn_b, dpdn_c):
    dff = df[:200]
    if dpdn_a is None or dpdn_b is None or dpdn_c is None:
        pie_fig = px.pie(dff, names=dff.index, values='Street_robbery', title='Street Robbery Berlin')\
            .update_layout(showlegend=False, title_x=0.5).update_traces(textposition='inside',  textinfo='label+percent')
        pie_fig2 = px.pie(dff, names=dff.index, values='Drugs', title='Drugs Berlin')\
            .update_layout(showlegend=False, title_x=0.5).update_traces(textposition='inside', textinfo='label+percent')
        return pie_fig, pie_fig2
    else:
        raise jupyter_dash.exceptions.PreventUpdate

In [7]:
app.run_server(mode='inline')